# Prepare a Sample Corpus

Let’s now take a sample corpus of documents on which we will run most of our analyses in this article. A corpus is typically a collection of text documents usually belonging to one or more subjects or domains.

In [1]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')

     |████████████████████████████████| 283 kB 5.2 MB/s 
     |████████████████████████████████| 321 kB 60.4 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85437 sha256=931e3479f5d1d279ab9add0b35fa1af0b55f469f85e98aaaa7c72c9eb11d5977
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 200

corpus = ['The sky is blue and beautiful sky sky',
          'Love this blue and beautiful sky blue blue!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful sky sky,weather
1,Love this blue and beautiful sky blue blue!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beautiful today,weather
7,The dog is lazy but the brown fox is quick!,animals


In [3]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import nltk 
import re

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(text):
  text = re.sub(r'[^a-zA-Z\s]','',text,re.I)
  text = text.lower()
  text = text.strip()
  tokens = nltk.word_tokenize(text)
  filtered_tokens = [token for token in tokens if token not in stop_words]
  return  " ".join(filtered_tokens)

In [5]:
corpus

array(['The sky is blue and beautiful sky sky',
       'Love this blue and beautiful sky blue blue!',
       'The quick brown fox jumps over the lazy dog.',
       "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
       'I love green eggs, ham, sausages and bacon!',
       'The brown fox is quick and the blue dog is lazy!',
       'The sky is very blue and the sky is very beautiful today',
       'The dog is lazy but the brown fox is quick!'], dtype='<U66')

In [6]:
normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful sky sky', 'love blue beautiful sky blue blue',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham , bacon , eggs , toast beans',
       'love green eggs ham sausages bacon !',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'], dtype='<U57')

In [10]:
!pip install contractions 
!pip install textsearch 
!pip install tqdm
import nltk

In [11]:
dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2', compression='bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [12]:
dataset.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [13]:
#lexicon based sentiment analysis models - Use the model to classify
dataset['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [14]:
dataset.shape

(50000, 2)

In [15]:
#Split it into train and test
reviews = dataset['review'].values
sentiments = dataset['sentiment'].values


train_reviews = reviews[:35000]
test_reviews = reviews[35000:]

train_sentiments = sentiments[:35000]
test_sentiments = sentiments[35000:]

In [8]:
#Perform pre-processing to the text
import contractions
from bs4 import BeautifulSoup 
import tqdm
import unicodedata
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
  norm_docs = []
  for text in tqdm.tqdm(docs): 
    text = strip_html_tags(text)
    text = re.sub(r'[^a-zA-Z\s]','',text,re.I)
    text = text.lower()
    text = text.strip()
    text = remove_accented_chars(text)
    text = contractions.fix(text)
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    text = " ".join(filtered_tokens)
    text = re.sub(" +", ' ',text)
    text =text.strip()
    norm_docs.append(text)
  return norm_docs

In [ ]:
%%time 

norm_train_corpus = pre_process_corpus(train_reviews)
norm_test_corpus = pre_process_corpus(test_reviews)

100%|██████████| 15000/15000 [00:35<00:00, 422.33it/s]

CPU times: user 1min 59s, sys: 822 ms, total: 2min
Wall time: 2min


In [17]:
import tensorflow as tf

t = tf.keras.preprocessing.text.Tokenizer(oov_token= '<UNK>')#If any word not in the vocab is there name tag it UNK
# fit the tokenizer on the document 
t.fit_on_texts(norm_train_corpus)
t.word_index['<PAD>']= 0

In [18]:
train_sequences = t.texts_to_sequences(norm_train_corpus)
test_sequences = t.texts_to_sequences(norm_test_corpus)

In [19]:
print("Vocabulary size ={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size =97894
Number of Documents=35000


In [20]:
MAX_SEQUENCE_LENGTH = 1000

In [21]:
#pad dataset to a maximum review length in words
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [22]:
X_train.shape

(35000, 1000)

In [23]:
#Encode the labels 
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
num_classes = 2

In [24]:
y_train = le.fit_transform(train_sentiments)
y_test = le.transform(test_sentiments)

In [25]:
VOCAB_SIZE = len(t.word_index)

In [26]:
EMBEDDING_DIM = 300 #Dimension for dense embedding for each token

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim =VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.GRU(128,return_sequences = False))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer="adam",metrics =['accuracy'])

In [27]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 300)         29368200  
_________________________________________________________________
gru (GRU)                    (None, 128)               165120    
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 29,566,601
Trainable params: 29,566,601
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X_train,y_train,epochs = 2,batch_size = 100,shuffle = True,validation_split =0.1)

Epoch 1/2
315/315 [==============================] - 137s 413ms/step - loss: 0.3715 - accuracy: 0.8279 - val_loss: 0.2791 - val_accuracy: 0.8854
Epoch 2/2
315/315 [==============================] - 129s 411ms/step - loss: 0.1365 - accuracy: 0.9514 - val_loss: 0.3101 - val_accuracy: 0.8777


In [29]:
## Evaluation on Test
scores = model.evaluate(X_test,y_test)
print("Accuracy: ",scores[1]*100)

469/469 [==============================] - 15s 31ms/step - loss: 0.2773 - accuracy: 0.8926
Accuracy:  89.25999999046326


In [30]:
predictions = model.predict_classes(X_test).ravel()
predictions = ['positive' if item==1 else 'negative' for item in predictions]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [31]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ['negative','positive']

print(classification_report(test_sentiments,predictions))
pd.DataFrame(confusion_matrix(test_sentiments,predictions),index = labels,columns = labels)

              precision    recall  f1-score   support

    negative       0.90      0.88      0.89      7490
    positive       0.88      0.90      0.89      7510

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



,negative,positive
negative,6599,891
positive,720,6790


In [40]:
#Bidirectional LSTM 
EMBEDDING_DIM = 300 #Dimension for dense embedding for each token

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim =VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences = False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer="adam",metrics =['accuracy'])

In [41]:
model.fit(X_train,y_train,epochs = 2,batch_size = 100,shuffle = True,validation_split =0.1)

Epoch 1/2
315/315 [==============================] - 170s 532ms/step - loss: 0.3567 - accuracy: 0.8403 - val_loss: 0.2797 - val_accuracy: 0.8911
Epoch 2/2
315/315 [==============================] - 167s 531ms/step - loss: 0.1265 - accuracy: 0.9553 - val_loss: 0.3190 - val_accuracy: 0.8837


In [42]:
## Evaluation on Test
scores = model.evaluate(X_test,y_test)
print("Accuracy: ",scores[1]*100)

469/469 [==============================] - 29s 59ms/step - loss: 0.3009 - accuracy: 0.8869
Accuracy:  88.69333267211914


In [43]:
#STacked idirectional LSTM 
EMBEDDING_DIM = 300 #Dimension for dense embedding for each token

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim =VOCAB_SIZE,output_dim = EMBEDDING_DIM,input_length = MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences = True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences = False)))
model.add(tf.keras.layers.Dense(256,activation = 'relu'))
model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',optimizer="adam",metrics =['accuracy'])

In [44]:
model.fit(X_train,y_train,epochs = 2,batch_size = 100,shuffle = True,validation_split =0.1)

Epoch 1/2
315/315 [==============================] - 327s 1s/step - loss: 0.3618 - accuracy: 0.8385 - val_loss: 0.3272 - val_accuracy: 0.8811
Epoch 2/2
315/315 [==============================] - 323s 1s/step - loss: 0.1474 - accuracy: 0.9490 - val_loss: 0.3295 - val_accuracy: 0.8737


In [45]:
## Evaluation on Test
scores = model.evaluate(X_test,y_test)
print("Accuracy: ",scores[1]*100)

469/469 [==============================] - 67s 140ms/step - loss: 0.3064 - accuracy: 0.8809
Accuracy:  88.09333443641663
